In [1]:
import cv2
import pandas as pd
import numpy as np
import random

def preprocessing(data, labels, val=True):
    
    data = pd.read_csv(data)
    labels = pd.read_csv(labels)

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [4]:
X_train, y_train = preprocessing(data='data/x_train_gr_smpl.csv', labels='data/y_train_smpl.csv')
X_train['label'] = y_train
X_train = X_train.sample(frac=1)

X_test, y_test = preprocessing(data='data/x_test_gr_smpl.csv', labels='data/y_test_smpl.csv')
X_test['label'] = y_test
X_test = X_test.sample(frac=1)

y_train = X_train['label']
X_train = X_train.iloc[:, :-1]

y_test = X_test['label']
X_test = X_test.iloc[:, :-1]

In [5]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Using TensorFlow backend.


In [6]:
IMG_SIZE = 48

X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(X_train[0][0][0])

X_train = X_train/255.0
X_test = X_test/255.0

print(X_train[0][0][0])
print(X_train.shape[1:])

[32]
[0.1254902]
(48, 48, 1)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import time


dense_layers = [1]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:

#             NAME = f"{conv_layer}-conv-{layer_size}-nodes-{dense_layer}-dense-{int(time.time())}"
#             tensorboard = TensorBoard(log_dir=f"logs\{NAME}")

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X_train.shape[1:], activation='relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3), activation='relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
                model.add(Dropout(0.2))

            model.add(Dense(10, activation='softmax'))

            model.compile(loss="categorical_crossentropy",
                         optimizer="adam",
                         metrics=["accuracy"])

            model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20) # , callbacks=[tensorboard])
            
# model.save('6ax3-CNN.model')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 12660 samples, validate on 4170 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
12660/12660 [==============================] - 17s 1ms/sample - loss: 0.5862 - acc: 0.7897 - val_loss: 0.3197 - val_acc: 0.8957
Epoch 2/20
12660/12660 [==============================] - 14s 1ms/sample - loss: 0.1802 - acc: 0.9311 - val_loss: 0.2162 - val_acc: 0.9170
Epoch 3/20
12660/12660 [==============================] - 14s 1ms/sample - loss: 0.1131 - acc: 0.9576 - val_loss: 0.2442 - val_acc: 0.9163
Epoch 4/20
12660/12660 [==============================] - 14s 1ms/sample - loss: 0.0803 - acc: 0.9698 - val_loss: 0.1952 - val_acc: 0.9314
Epoch 5/20
12660/12660 [==============================] - 14s 1ms/sample - loss: 0.0592 - acc: 0.9783 - val_loss: 0.2459 - val_acc: 0.9324
Epoch 6/20
12660/1266

In [ ]:
# model = tf.keras.models.load_model("6ax3-CNN.model")

test = 
prediction = model.predict('')